<a href="https://colab.research.google.com/github/stephenbeckr/numerical-analysis-class/blob/master/Demos/Ch2_AitkenExtrapolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aitken Extrapolation

## Task 1: recreate Table 2.10 in the book
In the table, we have $x_n = \cos(\frac{1}{n})$ which converges to $\cos(0)=1$ since $\cos$ is continuous.  Use Aitken Extrapolation to accelerate this sequence $(x_n)$.

In [2]:
# Recreate Table 2.10 in the book

import numpy as np
from numpy import diff as D   # this is the Delta in the book, aka "forward difference"

N = 7  # total number of terms
p = np.cos( 1/np.r_[1:N+1] ) # Example in book (Table 2.10)
print("The sequence p_n is:")
for pn in p:
  print("{:.5f}".format(pn))

print("The sequence \hat{p}_n is:")  
# Define phat (for p^hat, not as in "that is a phat song!")
phat = p[:-2]  - (D(p)[:-1])**2 / D(D(p))
for pn in phat:
  print("{:.5f}".format(pn))

The sequence p_n is:
0.54030
0.87758
0.94496
0.96891
0.98007
0.98614
0.98981
The sequence \hat{p}_n is:
0.96178
0.98213
0.98979
0.99342
0.99541


## Task 2: accelerate a root-finding problem
You could do Steffensen's variant, but we didn't discuss that, so I suggest you just to Aitken extrapolation

Let's solve $x = g(x)$ where
$$g(x) = .7\cos(x)$$
insdie the interval $[0,1]$.  We know that $g$ is a contraction since
$$|g'(x)| = |-.7\sin(x)| \le .7 < 1$$

In [3]:
# Try a root-finding problem, x = .7*cos(x)
from scipy.optimize import root_scalar
g = lambda x : .7*np.cos(x)

output = root_scalar(lambda x : g(x) - x ,bracket=[0,1])
trueRoot = output.root
print("The root is",trueRoot)

The root is 0.5839889663069239


In [4]:
# Accelerate the iteration

# Try fixed-point iteration, starting at 0
x = 0
N   = 20
p   = np.zeros(N)
phat= np.zeros(N-2)
# There are many ways to program this, depending on how/if you want to save
#   all the iterates.
for i in range(N):
  x = g(x)
  p[i] = x
  if i > 1:
    d1_a = p[i-1]-p[i-2]
    d1_b = p[i]-p[i-1]
    phat[i-2] = p[i-2] - (d1_a**2)/( d1_b - d1_a)
    print("p_{:<2d} is {:.5f} and error is {:.3e}; hat(p)_{:<2d} is {:.5f} and error is {:.3e}".format( i, x,abs(x-trueRoot),i-2,phat[i-2],abs(phat[i-2]-trueRoot) ) )
  else:
    print("p_{:<2d} is {:.5f} and error is {:.3e}".format( i, x,abs(x-trueRoot) ) )

p_0  is 0.70000 and error is 1.160e-01
p_1  is 0.53539 and error is 4.860e-02
p_2  is 0.60205 and error is 1.806e-02; hat(p)_0  is 0.58284 and error is 1.153e-03
p_3  is 0.57692 and error is 7.065e-03; hat(p)_1  is 0.58380 and error is 1.874e-04
p_4  is 0.58670 and error is 2.712e-03; hat(p)_2  is 0.58396 and error is 2.675e-05
p_5  is 0.58294 and error is 1.049e-03; hat(p)_3  is 0.58398 and error is 4.045e-06
p_6  is 0.58439 and error is 4.045e-04; hat(p)_4  is 0.58399 and error is 5.989e-07
p_7  is 0.58383 and error is 1.562e-04; hat(p)_5  is 0.58399 and error is 8.941e-08
p_8  is 0.58405 and error is 6.027e-05; hat(p)_6  is 0.58399 and error is 1.331e-08
p_9  is 0.58397 and error is 2.326e-05; hat(p)_7  is 0.58399 and error is 1.983e-09
p_10 is 0.58400 and error is 8.977e-06; hat(p)_8  is 0.58399 and error is 2.953e-10
p_11 is 0.58399 and error is 3.465e-06; hat(p)_9  is 0.58399 and error is 4.397e-11
p_12 is 0.58399 and error is 1.337e-06; hat(p)_10 is 0.58399 and error is 6.531e-1

In [5]:
# Same thing, but using a different version of the code
#   (designed to be simpler, since it doesn't save the entire history)
x   = 0  # our starting guess
for i in range(10): # i = 0, 1, 2, ..., 9 -- don't forget that Python is 0 based
  if i >= 1:
    xVeryOld = xOld
  xOld = x
  x    = g(x)  # Fixed point iteration update
  if i >= 1:
      DeltaOld = Delta
  Delta = x - xOld
  if i >= 1:
    xhat = xVeryOld - DeltaOld**2/( Delta - DeltaOld)
    print(xVeryOld,xhat)

0 0.5667292007496922
0.7 0.5828355747666369
0.5353895310991419 0.5838016125355683
0.6020489755511681 0.5839622114962584
0.5769238612151603 0.583984921687019
0.5867011411648845 0.5839883673835934
0.5829400576356008 0.5839888768922955
0.5843934705791918 0.5839889529996376
0.5838328003801305 0.5839889643240639
